<a href="https://colab.research.google.com/github/raheelam98/langgraph_guru/blob/main/agent_basic_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# packages:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain_google_genai langchain   python-dotenv

In [2]:
# API keys set up
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "documents"


In [3]:
import os
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

In [ ]:
# llm
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    api_key=gemini_api_key
)

In [ ]:
# packages
%%capture --no-stderr
%pip install -U tavily-python langchain_community

In [ ]:
# API keys set up
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

**Lang Chain and Lang Graph Documents**

In [ ]:
from typing import List
from langchain_core.messages import ToolMessage,BaseMessage